In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from playsound import playsound
import threading
import time

start_away_time = None
alarm_triggered = False

def play_siren():
    playsound("civil-defense-siren-128262.mp3")

cap = cv2.VideoCapture(0)
mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(refine_landmarks=True)


while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    h,w,_ = f.shape

    res = face_mesh.process(rgb)

    

    if res.multi_face_landmarks:
        for face_landmarks in res.multi_face_landmarks:
            nose_tip = face_landmarks.landmark[1]
            left_eye = face_landmarks.landmark[33]
            right_eye = face_landmarks.landmark[263]

            x_nose = int(nose_tip.x*w)
            x_left = int(left_eye.x*w)
            x_right = int(right_eye.x*w)

            cv2.circle(f,(x_nose,int(nose_tip.y*h)),5,(0,255,0),-1)
            cv2.circle(f,(x_left,int(left_eye.y*h)),5,(0,255,0),-1)
            cv2.circle(f,(x_right,int(right_eye.y*h)),5,(0,255,0),-1)

            nose_x = x_nose
            left_dist = abs(nose_x-x_left)
            right_dist = abs(x_right-nose_x)

            threshold = 60
            # print(left_dist,right_dist)
            if left_dist>threshold and left_dist>right_dist:
                direction = "Looking Right"
                focused=False
            
            elif right_dist>threshold and right_dist>left_dist:
                direction = "Looking Left"
                focused=False
            else:
                direction = "Looking Center"
                focused=True

            cv2.putText(f,direction,(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)

        if not focused:
            if start_away_time is None:
                start_away_time = time.time()
            elif time.time() - start_away_time > 20 and not alarm_triggered:
                threading.Thread(target=play_siren).start()
                alarm_triggered=True
        else:
            start_away_time=None
            alarm_triggered=False

        if not focused and start_away_time:
            elapsed = int(time.time()-start_away_time)
            cv2.putText(f,f"Away for: {elapsed}",(10,60),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,255),2)
            


    cv2.imshow("FOCUS DETECTION",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()